### MAST30034: Applied Data Science Project 1
---
# Preprocessing Part 1: Cleaning The Data
#### Xavier Travers (1178369)

Cleaning the datasets of null, inconsistent, or unnecessary values.
This is performed on the TLC data and COVID data.

In [1]:
# imports used throughout this notebook
from collections import defaultdict
from itertools import product
import os
import sys
from pyspark.sql import DataFrame, Column
from pyspark.sql.types import IntegerType
from pyspark.sql import functions as F

# add homemade helpers
sys.path.insert(1, '../scripts')
import helpers.cleaning_helpers as ch
import helpers.join_helpers as jh

# Used for saving time (if you don't want sanity-check printouts)
INTERMEDIATE_OUTPUTS = False

In [2]:
from pyspark.sql import SparkSession

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName('MAST30034 XT Project 1')
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.sql.repl.eagerEval.enabled', True) 
    .config('spark.sql.parquet.cacheMetadata', 'true')
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

22/08/14 00:21:47 WARN Utils: Your hostname, Polaris resolves to a loopback address: 127.0.1.1; using 172.26.235.73 instead (on interface eth0)
22/08/14 00:21:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/14 00:21:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/08/14 00:21:49 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/08/14 00:21:49 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/08/14 00:21:49 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/08/14 00:21:49 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [3]:
# import the cdc week file to convert all dates to cdc weeks now
mmwr_weeks_df = spark.read.parquet('../data/raw/virals/mmwr_weeks.parquet')
mmwr_weeks_df.limit(5)

year,month,day,cdc_week,week_index,us_format,week_ending,week_month,week_year,timeline
2017,12,31,1,1,12/31/2017,2018-01-06,1,2018,neither
2018,1,1,1,1,01/01/2018,2018-01-06,1,2018,neither
2018,1,2,1,1,01/02/2018,2018-01-06,1,2018,neither
2018,1,3,1,1,01/03/2018,2018-01-06,1,2018,neither
2018,1,4,1,1,01/04/2018,2018-01-06,1,2018,neither


In [4]:
# import the zones dataset
zones_df = spark.read.csv('../data/raw/tlc_zones/zones.csv',
    header = True)

### 1. Cleaning the TLC dataset(s)

In [5]:
if INTERMEDIATE_OUTPUTS:
    example_df = spark.read.parquet('../data/raw/tlc/yellow/2019-07.parquet/')
    example_df.limit(5)
# TODO: commenting

In [6]:
if INTERMEDIATE_OUTPUTS:
    example_df.sort('trip_distance', ascending = False).limit(5)

In [7]:
# names of the tlc datasets to clean 
# (I was originally planning on working on fhvhv and green as well)
TLC_NAMES = ['yellow']

# dictionary to rename all the columns I want to keep
TLC_KEEP_COLUMNS = {
    'tpep_pickup_datetime': 'date',
    'passenger_count': 'passengers',
    'trip_distance': 'trip_distance',
    'PULocationID': 'pu_location_id',
    'DOLocationID': 'do_location_id',
    'hours_elapsed': 'hours_elapsed'
    # #  below only apply to fhvhv
    # 'hvfhs_license_num': 'fhvhv_license',
    # 'pickup_datetime': 'date',
    # 'trip_miles': 'trip_distance',
    # 'shared_request_flag': 'shared'
}

# create a dictionary of the columns to keep and the required filters
TLC_CLEAN_COLUMNS = {
    'pu_location_id': [ch.non_null], 
    'do_location_id': [ch.non_null], 
    'passengers': [ch.non_null], 
    'trip_distance': [ch.non_null, ch.non_negative], 
    # 'fhvhv_license': [ch.non_null], 
}

In [8]:
# read in the tlc data
tlc_df = jh.read_stacked_tlc_df(spark)

In [9]:
if INTERMEDIATE_OUTPUTS:
    # sanity check
    tlc_df.limit(5)

In [10]:
# derive extra values which are used to filter out valid trips
SECONDS_TO_HOURS = 1 / (60*60)
tlc_df = tlc_df\
    .withColumn('hours_elapsed', 
        (
            (F.col("tpep_dropoff_datetime").cast("long")
            - F.col('tpep_pickup_datetime').cast("long")) 
            * SECONDS_TO_HOURS
        )
    )\
    .withColumn('mph', (F.col('trip_distance') / F.col('hours_elapsed')))

In [11]:
if INTERMEDIATE_OUTPUTS:
    tlc_df.limit(5) 

In [12]:
# https://ypdcrime.com/vt/article30.php?zoom_highlight=fifty+five+miles+per+hour#t1180-a.
# As per: https://www.dot.ny.gov/divisions/operating/oom/transportation-systems/repository/TSMI-17-05.pdf
# the NYS maximum speed limit is 65 mph. filter out trips faster than legal.
tlc_df = tlc_df.where(
    (F.col('mph').isNotNull()) &
    (F.col('mph') <= 65)
)

In [13]:
if INTERMEDIATE_OUTPUTS:
    # this one is time instensive 
    tlc_df.sort('trip_distance', ascending = False).limit(5)

In [14]:
tlc_df = ch.perform_cleaning(tlc_df, mmwr_weeks_df, TLC_KEEP_COLUMNS, 
    TLC_CLEAN_COLUMNS)

AnalysisException: Column 'week_index' does not exist. Did you mean one of the following? [week_ending, week_month, week_year, timeline, date, day, month, passengers, year, trip_distance, hours_elapsed, do_location_id, pu_location_id];
'Project [year#2470, month#2453, day#2461, week_ending#6, week_year#8L, week_month#7L, 'week_index, timeline#9, date#2446, passengers#2411, trip_distance#2418, pu_location_id#2425L, do_location_id#2432L, hours_elapsed#2439]
+- Project [week_ending#6, week_year#8L, week_month#7L, timeline#9, date#2446, passengers#2411, trip_distance#2418, pu_location_id#2425L, do_location_id#2432L, hours_elapsed#2439, month#2453, day#2461, year#2470]
   +- Project [day#2461, month#2453, year#2470, date#2446, passengers#2411, trip_distance#2418, pu_location_id#2425L, do_location_id#2432L, hours_elapsed#2439, cdc_week#3L, week_index#4L, us_format#5, week_ending#6, week_month#7L, week_year#8L, timeline#9]
      +- Join Inner, (((cast(day#2461 as bigint) = day#2L) AND (cast(m

In [ ]:
# next, filter out trips which do not start and/or end within the 5 boroughs 
tlc_df = ch.extract_borough_name(tlc_df, zones_df,  'pu')
tlc_df = ch.extract_borough_name(tlc_df, zones_df,  'do')

if INTERMEDIATE_OUTPUTS:
    tlc_df.sort('trip_distance', ascending=False).limit(5)

In [ ]:
if INTERMEDIATE_OUTPUTS:
    tlc_df.count()

In [ ]:
if INTERMEDIATE_OUTPUTS:
    tlc_df.sort('week_index', ascending = False).limit(10)

In [ ]:
if INTERMEDIATE_OUTPUTS:
    tlc_df.sort('trip_distance', ascending=False).limit(5)

In [ ]:
# filter by anything outside of the time bounded by the timelines
# (keep the transitionary time (2019) for time-series analysis)
tlc_df = tlc_df.where(F.col('timeline') != 'neither')

In [ ]:
# save the stacked df by month (this will take a while)
tlc_df = tlc_df.sort('week_year', 'week_month')
tlc_df.write\
    .partitionBy('week_year', 'week_month')\
    .mode('overwrite')\
    .parquet(f'../data/curated/tlc/cleaned/yellow')

### 2. Cleaning the COVID dataset

In [ ]:
# read in the covid dataset
covid_df = spark.read.csv('../data/raw/virals/covid/cases-by-day.csv',
    header = True)
covid_df.limit(5)
# TODO: commenting

In [ ]:
# sum the number of incomplete datasets (ensure no incomplete values)
if INTERMEDIATE_OUTPUTS:
    sum(covid_df.select('INCOMPLETE'))
# TODO: commenting

In [ ]:
# TODO: commenting on covid cleaning
COVID_KEEP_COLUMNS = {
    'date_of_interest':'date'
}

COVID_CLEAN_COLUMNS = defaultdict(lambda: ch.non_negative)

COVID_BOROUGHS = {
    'BX_':'Bronx',
    'BK_':'Brooklyn',
    'MN_':'Manhattan',
    'QN_':'Queens',
    'SI_':'Staten Island',
}

COVID_COUNTS = {
    'CASE_COUNT': 'cases', 
    'DEATH_COUNT': 'deaths', 
    'HOSPITALIZED_COUNT': 'hospitalised'
}
# TODO: commenting
for prefix, new_prefix in COVID_BOROUGHS.items():
    for suffix, new_suffix in COVID_COUNTS.items():
        COVID_KEEP_COLUMNS[f'{prefix}{suffix}'] = f'{new_prefix}{new_suffix}'

In [ ]:
covid_df = ch.perform_cleaning(covid_df, mmwr_weeks_df, COVID_KEEP_COLUMNS, 
    COVID_CLEAN_COLUMNS)
# TODO: commenting

In [ ]:
from itertools import product

temp_df = None
# TODO: commenting
COVID_DATE_COLUMNS = [
    F.col('date'), 
    F.col('week_ending'), 
    F.col('week_year'), 
    F.col('week_month'), 
    F.col('week_index')
    F.col('timeline')
]

# The data here is very wide, I'd rather just have a 'borough' column
# for homogeneity of all the data
for prefix in COVID_BOROUGHS.values():
    borough_columns = []
    for suffix in COVID_COUNTS.values():
        borough_columns.append(F.col(f'{prefix}{suffix}').alias(suffix))

    if temp_df == None:
        temp_df = covid_df.select(COVID_DATE_COLUMNS + borough_columns)\
            .withColumn('borough', F.lit(prefix))
    else:
        temp_df = temp_df\
            .union(
                covid_df.select(COVID_DATE_COLUMNS + borough_columns)\
                    .withColumn('borough', F.lit(prefix))
            )
    
covid_df = temp_df

In [ ]:
if INTERMEDIATE_OUTPUTS:
    covid_df.sort('week_index', 'date').limit(5)
# TODO: commenting

In [ ]:
# save the cleaned covid data
# TODO: commenting
covid_df.write.mode('overwrite').parquet('../data/curated/virals/covid/cleaned/cases-by-day')

### 3. Cleaning the flu dataset

In [ ]:
# read in the flu dataset
# TODO: commenting
flu_df = spark.read.csv('../data/raw/virals/flu/cases-by-week.csv',
    header=True)
flu_df.limit(5)

In [ ]:
FLU_KEEP_COLUMNS = {
    'Week Ending Date': 'date',
    'Region': 'region',
    'County': 'borough',
    'Disease': 'disease',
    'Count': 'cases',
}
# TODO: commenting
FLU_CLEAN_COLUMNS = {
    'date': [],
    'region': [lambda _: F.col('region') == 'NYC'],
    'borough': [],
    'disease': [],
    'cases': [ch.non_negative]
}

In [ ]:
# TODO: commenting
flu_df:DataFrame = ch.perform_cleaning(flu_df, mmwr_weeks_df, FLU_KEEP_COLUMNS, 
    FLU_CLEAN_COLUMNS)

In [ ]:
if INTERMEDIATE_OUTPUTS:
    # get the list of distinct counties (column now called 'borough')
    flu_df.select('borough').distinct().limit(5)

In [ ]:
# map the boroughs to their proper names
# from: https://portal.311.nyc.gov/article/?kanumber=KA-02877
# also from map dict
FLU_COUNTY_TO_BOROUGH = {
    'BRONX': 'Bronx',
    'KINGS': 'Brooklyn',
    'NEW YORK': 'Manhattan',
    'QUEENS': 'Queens',
    'RICHMOND': 'Staten Island'
}

In [ ]:
# apply the mapping to the flu df
flu_df = ch.replace_column_using_dict(flu_df, 'borough', FLU_COUNTY_TO_BOROUGH)

# also remove the regions column (not needed anymore)
columns_without_regions = flu_df.columns[:]
columns_without_regions.remove('region')
flu_df = flu_df.select(columns_without_regions)

In [ ]:
if INTERMEDIATE_OUTPUTS:
    flu_df.limit(5)

In [ ]:
# save the cleaned flu data
flu_df.write.mode('overwrite').parquet('../data/curated/virals/flu/cleaned/cases-by-week')